In [1]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

In [2]:
data = Dataset.load_builtin('ml-100k',prompt=False)
data.raw_ratings[:10]
#user , item , rate , time

[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013')]

In [3]:
model = SVD()

In [4]:
cross_validate(model,data,measures=['rmse','mae'],cv=5,verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9347  0.9319  0.9419  0.9359  0.9405  0.9370  0.0037  
MAE (testset)     0.7373  0.7340  0.7416  0.7385  0.7385  0.7380  0.0024  
Fit time          5.69    4.83    5.10    5.66    4.93    5.24    0.36    
Test time         0.16    0.21    0.21    0.22    0.23    0.20    0.02    


{'test_rmse': array([0.93471174, 0.93188609, 0.94192956, 0.93586356, 0.94046053]),
 'test_mae': array([0.73732116, 0.73400318, 0.74159673, 0.73854139, 0.73852006]),
 'fit_time': (5.694603204727173,
  4.8284831047058105,
  5.097336292266846,
  5.65665340423584,
  4.934829235076904),
 'test_time': (0.15955305099487305,
  0.20847082138061523,
  0.2074441909790039,
  0.21741795539855957,
  0.23135948181152344)}

In [5]:
import numpy as np
from surprise import Dataset

In [6]:
data  = Dataset.load_builtin('ml-100k', prompt=False)
raw_data = np.array(data.raw_ratings, dtype=int)

In [7]:
raw_data[:,0] -= 1
raw_data[:,1] -= 1

In [8]:
n_users = np.max(raw_data[:,0])
n_movies = np.max(raw_data[:,1])
shape = (n_users+1, n_movies+1)
shape

(943, 1682)

In [9]:
adj_matrix = np.ndarray(shape,dtype=int)
for user_id,movie_id,rating,time in raw_data:
    adj_matrix[user_id][movie_id] = 1.
adj_matrix # 인접 행렬

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [10]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1,-1, []

for user_id,user_vector in enumerate(adj_matrix):
    if my_id != user_id:
        similarity = np.dot(my_vector,user_vector)
        if similarity > best_match:
            best_match = similarity
            best_match_id = user_id
            best_match_vector = user_vector

print('Best Match: {},Best_Match_ID: {}'.format(best_match, best_match_id))

Best Match: 183,Best_Match_ID: 275


In [11]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
    log1, log2 = log
    if log1 < 1. and log2 > 0.:
        recommend_list.append(i)
        
# 내가 보지 않고 275(best match id)이 본 영화 추천
print(recommend_list)

[272, 273, 275, 280, 281, 283, 287, 288, 289, 290, 292, 293, 297, 299, 300, 301, 302, 306, 312, 314, 315, 316, 317, 321, 322, 323, 324, 327, 330, 331, 332, 333, 339, 342, 345, 346, 353, 354, 355, 356, 357, 363, 364, 365, 366, 372, 374, 378, 379, 381, 382, 383, 384, 385, 386, 387, 390, 391, 392, 394, 395, 396, 398, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 412, 414, 416, 417, 418, 419, 420, 422, 424, 425, 426, 427, 428, 430, 431, 432, 435, 442, 446, 447, 448, 449, 450, 451, 452, 454, 455, 457, 460, 461, 462, 468, 469, 470, 471, 472, 473, 474, 478, 495, 500, 507, 517, 522, 525, 530, 539, 540, 543, 545, 546, 548, 549, 550, 551, 553, 557, 558, 560, 561, 562, 563, 565, 566, 567, 568, 570, 571, 574, 575, 576, 577, 580, 581, 582, 585, 587, 589, 590, 594, 596, 602, 623, 626, 627, 630, 633, 635, 639, 646, 648, 651, 652, 654, 657, 664, 668, 671, 677, 678, 681, 683, 684, 685, 690, 691, 692, 695, 696, 708, 709, 714, 718, 719, 720, 724, 726, 727, 731, 733, 734, 736, 738, 741, 742, 745,

In [ ]:
# 유크릴드 거리를 사용한 추천
# 거리가 가까울수록(값이 작을수록) 나와 유사한 사용자

In [12]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999,-1, []

for user_id,user_vector in enumerate(adj_matrix):
    if my_id != user_id:
        # euclidean distance
        euclidean_dist = np.sqrt(np.sum(np.square(my_vector-user_vector)))
        if euclidean_dist < best_match:
            best_match = euclidean_dist
            best_match_id = user_id
            best_match_vector = user_vector

print('Best Match: {},Best_Match_ID: {}'.format(best_match, best_match_id))

Best Match: 14.832396974191326,Best_Match_ID: 737


In [13]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
    log1, log2 = log
    if log1 < 1. and log2 > 0.:
        recommend_list.append(i)
        
# 내가 보지 않고 275(best match id)이 본 영화 추천
print(recommend_list)

[297, 312, 317, 342, 356, 366, 379, 384, 392, 402, 404, 407, 417, 422, 428, 433, 448, 454, 469, 473, 495, 510, 516, 526, 527, 549, 567, 602, 635, 649, 650, 654, 658, 661, 664, 696, 731, 746, 750, 754, 915, 918, 925, 929, 950, 968, 1015, 1046]


In [ ]:
# 코사인 유사도